# dim_time

### Importación de librerías

In [ ]:
import yaml
import pandas as pd
from sqlalchemy import create_engine

### Conexión a base y bodega de datos

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlSource = f"{configSource['driver']}://{configSource['user']}:{configSource['password']}@{configSource['host']}:{configSource['port']}/{configSource['db']}"
urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineSource = create_engine(urlSource)
engineDestination = create_engine(urlDestination)

### Extracción y transformación de datos

In [ ]:

# Crear la columna `time` en formato datetime.time para que se almacene como hora en SQL
time = pd.DataFrame({"time": pd.date_range("2021-01-01", "2021-01-01 23:59", freq="T")})
time["hour"] = time["time"].dt.hour
time["minute"] = time["time"].dt.minute

# Convertir a datetime.time para asegurar que la base de datos lo guarde como hora
time["time"] = time["time"].dt.time

time["key_dim_time"] = range(1, len(time) + 1)
time["saved"] = pd.to_datetime("now")

time


### Carga de datos

In [ ]:
time.to_sql("dim_time", engineDestination, index=False, if_exists="replace")